# RecBERT2LM

可用的中文预训练参数：[`bert-base`](https://storage.googleapis.com/bert_models/2018_11_03/chinese_L-12_H-768_A-12.zip),[`roberta-wwm-ext-base`](https://drive.google.com/uc?export=download&id=1jMAKIJmPn7kADgD3yQZhpsqM-IRM1qZt),[`roberta-wwm-ext-large`](https://drive.google.com/uc?export=download&id=1dtad0FFzG11CBsawu8hvwwzU2R0FDI94),[`macbert-base`](https://drive.google.com/uc?export=download&id=1aV69OhYzIwj_hn-kO1RiBa-m8QAusQ5b),[`macbert-large`](https://drive.google.com/uc?export=download&id=1lWYxnk1EqTA2Q20_IShxBrCPc5VSDCkT)

In [1]:
import uf

print(uf.__version__)

v2.5.19


In [2]:
model = uf.RecBERT2LM("../../ref/bert_config.json", "../../ref/vocab.txt", max_seq_length=10)
print(model)

uf.RecBERT2LM(
    config_file="../../ref/bert_config.json",
    vocab_file="../../ref/vocab.txt",
    max_seq_length=10,
    init_checkpoint=None,
    output_dir=None,
    gpu_ids=None,
    add_prob=0.05,
    rep_prob=0.05,
    del_prob=0.05,
    do_lower_case=True,
    truncate_method="LIFO",
)


In [3]:
X = ["天亮以前说再见", "笑着泪流满面", "去迎接应该你的", "更好的明天", "他想知道那是谁", "为何总沉默寡言", "人群中也算抢眼", "抢眼的孤独难免"]

# 训练

In [4]:
model.fit(X, total_steps=20)

/Users/geyingli/Library/Python/3.8/lib/python/site-packages/uf-2.5.19-py3.8.egg/uf/apps/bert/bert.py:235: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  query_layer = tf.layers.dense(
/Users/geyingli/Library/Python/3.8/lib/python/site-packages/uf-2.5.19-py3.8.egg/uf/apps/bert/bert.py:245: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  key_layer = tf.layers.dense(
/Users/geyingli/Library/Python/3.8/lib/python/site-packages/uf-2.5.19-py3.8.egg/uf/apps/bert/bert.py:255: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  value_layer = tf.layers.dense(
/Users/geyingli/Library/Python/3.8/lib/python/site-packages/uf-2.5.19-py3.8.egg/uf/apps/bert/bert.py:379: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Ple

INFO:tensorflow:Build graph with 333,379,608 parameters (among which 111,126,536 are trainable)
INFO:tensorflow:Running local_init_op
INFO:tensorflow:Done running local_init_op
INFO:tensorflow:Running training on 8 samples (step 0 -> 20)
INFO:tensorflow:step 1, add_accuracy 0.5075, add_loss 0.729563, rep_accuracy 0.4030, rep_loss 0.826213, del_accuracy 0.4030, del_loss 0.810844, cls_accuracy 0.3750, cls_loss 0.710651, 0.11 steps/sec, 0.85 examples/sec
INFO:tensorflow:step 2, add_accuracy 0.9701, add_loss 0.204016, rep_accuracy 0.9701, rep_loss 0.177103, del_accuracy 1.0000, del_loss 0.123412, cls_accuracy 0.6250, cls_loss 0.718593, 0.54 steps/sec, 4.28 examples/sec
INFO:tensorflow:step 3, add_accuracy 0.9701, add_loss 0.117575, rep_accuracy 0.9701, rep_loss 0.116470, del_accuracy 1.0000, del_loss 0.015036, cls_accuracy 0.3750, cls_loss 0.834942, 0.65 steps/sec, 5.19 examples/sec
INFO:tensorflow:step 4, add_accuracy 0.9701, add_loss 0.109563, rep_accuracy 0.9701, rep_loss 0.127421, del_

# 推理

In [5]:
X = [
    "他想知道那是谁道", "为何总沉默寡", 
    "群人面群中也抢面", "抢孤为独道难免",
]           # 人为增加一些错误，看模型是否能识别并纠正 (与训练阶段采样不同，因此此case预测错误正常)
model.predict(X)

INFO:tensorflow:Running inference on 4 samples
INFO:tensorflow:process 100.0%, 3.35 examples/sec


{'lm_preds': ['他想知道那是谁道', '为何总沉默寡', '群人面群中也抢面', '抢孤为独道难免'],
 'cls_probs': array([[9.9934238e-01, 6.5759692e-04],
        [9.9661535e-01, 3.3846260e-03],
        [9.9592650e-01, 4.0734927e-03],
        [8.8263565e-01, 1.1736435e-01]], dtype=float32),
 'cls_preds': [0, 0, 0, 0]}

# 训练样本生成

In [14]:
X = ["天亮以前说再见", "笑着泪流满面", "去迎接应该你的", "更好的明天", "他想知道那是谁", "为何总沉默寡言", "人群中也算抢眼", "抢眼的孤独难免"]
data = model.convert(X, is_training=True)
for i in range(len(data["input_ids"])):
    print("input_tokens: %s" % (model.tokenizer.convert_ids_to_tokens(data["input_ids"][i])))
    print("add_label_ids: %s" % (model.tokenizer.convert_ids_to_tokens(data["add_label_ids"][i])))
    print("rep_label_ids: %s" % (model.tokenizer.convert_ids_to_tokens(data["rep_label_ids"][i])))
    print("del_label_ids: %s" % (data["del_label_ids"][i]))


input_tokens: ['[CLS]', '天', '亮', '中', '前', '说', '再', '见', '[SEP]', '[PAD]']
add_label_ids: ['[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']
rep_label_ids: ['[PAD]', '[PAD]', '[PAD]', '以', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']
del_label_ids: [0 0 0 0 0 0 0 0 0 0]
input_tokens: ['[CLS]', '笑', '着', '为', '流', '满', '面', '[SEP]', '[PAD]', '[PAD]']
add_label_ids: ['[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']
rep_label_ids: ['[PAD]', '[PAD]', '[PAD]', '泪', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']
del_label_ids: [0 0 0 0 0 0 0 0 0 0]
input_tokens: ['[CLS]', '去', '迎', '接', '该', '你', '的', '[SEP]', '[PAD]', '[PAD]']
add_label_ids: ['[PAD]', '[PAD]', '[PAD]', '应', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']
rep_label_ids: ['[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']
del_label_ids: [0 0 0 0 0 0 0 0 0 0]
input_tokens: ['[CLS]', '好